## This is the final part of aproximation

A week ago, one person was published a notebook where the log loss was 0.38, but the notebook couldn't be comitted and submitted due to the fact that there were no NN weights to detect fake/real faces

And I decided to try to find a pattern that would get this from my result.

Interpolation of Lagrange, Newton, Splines, Regressions, Maximum Likelihood Method, Statistical Modeling Method, Monte Carlo Method, Latent Dirichlet Variables, EM algorithm - nothing couldn't approximate my values with these other than trees that are fitting by gradient descent

When I was submit result by GBT Regressor I get 0.47, read some discussion and I was understand that my notebook must predict 4000 videos, not 400. Ahahahaahahah, it was a big fail.

Conclusion: 290 place in Public Leaderboard, this was great experience and good intro into Computer Vision :)

In [1]:
import numpy as np
import pandas as pd

In [2]:
Sub038 = pd.read_csv('data/Sub038.csv')

Sub038.sort_values(by = 'filename', inplace = True)

Sub038 = Sub038.reset_index(drop = True)

In [3]:
SubForReg = pd.read_csv('data/SubForReg.csv')

SubForReg['FitIntercept'] = 1

SubForReg

,filename,label,FitIntercept
0,aassnaulhq.mp4,0.975000,1
1,aayfryxljh.mp4,0.017793,1
2,acazlolrpz.mp4,0.825523,1
3,adohdulfwb.mp4,0.021289,1
4,ahjnxtiamx.mp4,0.840784,1
...,...,...,...
395,ztyvglkcsf.mp4,0.038536,1
396,zuwwbbusgl.mp4,0.070050,1
397,zxacihctqp.mp4,0.103279,1
398,zyufpqvpyu.mp4,0.799663,1


In [37]:
ForExcel = pd.DataFrame({'Filename': SubForReg.filename, 'FitIntercept': SubForReg.FitIntercept,
                        
                        'X': SubForReg.label, 'Y': Sub038.label})

ForExcel

,Filename,FitIntercept,X,Y
0,aassnaulhq.mp4,1,0.975000,0.900000
1,aayfryxljh.mp4,1,0.017793,0.100000
2,acazlolrpz.mp4,1,0.825523,0.900000
3,adohdulfwb.mp4,1,0.021289,0.100000
4,ahjnxtiamx.mp4,1,0.840784,0.900000
...,...,...,...,...
395,ztyvglkcsf.mp4,1,0.038536,0.100000
396,zuwwbbusgl.mp4,1,0.070050,0.100000
397,zxacihctqp.mp4,1,0.103279,0.230713
398,zyufpqvpyu.mp4,1,0.799663,0.900000


In [38]:
XMatrix = ForExcel[['FitIntercept', 'X']]

XMatrixTransposed = XMatrix.T

In [39]:
print(XMatrix.shape)

print(XMatrixTransposed.shape)

(400, 2)
(2, 400)


In [41]:
MultipleXAndXT = XMatrixTransposed.dot(XMatrix)

MultipleXAndXT

,FitIntercept,X
FitIntercept,400.000000,198.434064
X,198.434064,147.007224


In [42]:
InverseOfMultipleXAndXT = np.linalg.inv(MultipleXAndXT)

InverseOfMultipleXAndXT

array([[ 0.00756723, -0.01021444],
       [-0.01021444,  0.0205901 ]])

In [43]:
InverseMatrixAndXMatrixTransposed = np.dot(InverseOfMultipleXAndXT, XMatrixTransposed)

In [44]:
InverseMatrixAndXMatrixTransposed.shape

(2, 400)

In [45]:
Coef = np.dot(InverseMatrixAndXMatrixTransposed, Sub038.label)

In [46]:
Coef

array([0.04232102, 0.94819576])

In [47]:
New = SubForReg.label.apply(lambda x: x * 0.94819576 + 0.04232102)

np.corrcoef(New, Sub038.label)

array([[1.       , 0.8861486],
       [0.8861486, 1.       ]])

In [48]:
ForExcel

,Filename,FitIntercept,X,Y
0,aassnaulhq.mp4,1,0.975000,0.900000
1,aayfryxljh.mp4,1,0.017793,0.100000
2,acazlolrpz.mp4,1,0.825523,0.900000
3,adohdulfwb.mp4,1,0.021289,0.100000
4,ahjnxtiamx.mp4,1,0.840784,0.900000
...,...,...,...,...
395,ztyvglkcsf.mp4,1,0.038536,0.100000
396,zuwwbbusgl.mp4,1,0.070050,0.100000
397,zxacihctqp.mp4,1,0.103279,0.230713
398,zyufpqvpyu.mp4,1,0.799663,0.900000


In [172]:
from xgboost import XGBRegressor

Model = XGBRegressor(n_estimators = 1000)

x = np.array(ForExcel.X).reshape(-1, 1)

Model.fit(x, ForExcel.Y)

Preds = Model.predict(x)

np.corrcoef(Preds, ForExcel.Y)

[14:45:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


array([[1.        , 0.99612194],
       [0.99612194, 1.        ]])

In [182]:
from sklearn.ensemble import GradientBoostingRegressor

Model2 = GradientBoostingRegressor(n_estimators = 1000)

Model2.fit(x, ForExcel.Y)

Preds2 = Model2.predict(x)

np.corrcoef(Preds2, ForExcel.Y)

array([[1.        , 0.99708157],
       [0.99708157, 1.        ]])

In [199]:
import pickle

pickle.dump(Model, open("weights.dat", "wb"))

In [200]:
loaded_model = pickle.load(open("weights.dat", "rb"))

y_pred = loaded_model.predict(x)

[10:55:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [201]:
np.corrcoef(y_pred, ForExcel.Y)

array([[1.        , 0.99612194],
       [0.99612194, 1.        ]])

In [205]:
list4 = [18, 39, 41, 74, 93, 110, 131, 133, 135, 142, 163, 195, 218, 248, 252, 277, 300, 304, 383]

for i in list4:

    print(i, "   ", s.label[i], "    ",ForExcel.Y[i])

18     0.8849223892444111      0.15028826892375946
39     0.10245259724566796      0.3538503348827362
41     0.8866669234084573      0.17580270767211914
74     0.8866669234084573      0.9
93     0.12361052117213085      0.3626951575279236
110     0.10245259724566796      0.3734246790409088
131     0.8867115010891271      0.9
133     0.1050473810030351      0.9
135     0.887221431076303      0.9
142     0.3463096192328899      0.9
163     0.17078637853053988      0.9
195     0.32847626501174576      0.5631140470504761
218     0.13736151482753348      0.9
248     0.8865929326835311      0.9
252     0.8880896776880312      0.9
277     0.8880896776880312      0.9
300     0.8880896776880312      0.9
304     0.11674673866362922      0.9
383     0.16298526266069258      0.8452033996582031
